In [ ]:
!pip install --upgrade pip
!pip install gdown

In [ ]:
import gdown

train_url = 'https://drive.google.com/uc?id=1-4WfgSQLMIMxl-vrqDjZTMiwb1w3qiJq'
val_url = 'https://drive.google.com/uc?id=1w9CuqPi3DbvbCN9DFGwLPeyYsXJvbjzk'
#the problem is that the whole file train+val is too big for kaggle!
#as kaggle gives 4.9gb of stage and the dataset is 2.8 gb
#since we are using zipped file first.
#https://drive.google.com/open?id=1-4WfgSQLMIMxl-vrqDjZTMiwb1w3qiJq --training
#https://drive.google.com/open?id=1w9CuqPi3DbvbCN9DFGwLPeyYsXJvbjzk --validation
output_train = 'dataset_train.zip'
output_val = 'dataset_val.zip'
gdown.download(train_url, output_train, quiet=False)
#os.remove(file_name)

In [ ]:
import os
def create_dir(dir):
  if not os.path.exists(dir):
    os.makedirs(dir)
    print("Created Directory : ", dir)
    return
create_dir("/kaggle/working/data_aug/train")
!unzip -q /kaggle/working/dataset_train.zip -d /kaggle/working/data_aug/train

In [ ]:
os.listdir("/kaggle/working/")

In [ ]:
file_name="/kaggle/working/dataset_train.zip"
os.remove(file_name)

In [ ]:
os.listdir("/kaggle/working/")

In [ ]:
gdown.download(val_url, output_val, quiet=False)

In [ ]:
os.listdir("/kaggle/working/")

In [ ]:
create_dir("/kaggle/working/data_aug/val")
!unzip -q /kaggle/working/dataset_val.zip -d /kaggle/working/data_aug/val

In [ ]:
os.listdir("/kaggle/working/data_aug")
file_name="/kaggle/working/dataset_val.zip"
os.remove(file_name)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Hello everyone!! Hope everything is fine and you are enjoying things on Kaggle as usual. The rage for competing on Kaggle should never end. 
Machine Learning and Deep Learning have a huge scope in healthcare but applying them in healthcare isn't that simple. The stake is very high. It's more than just a `classification` problem. But if applied very carefully, it can benefit the world in enormous ways. **And as a Machine learning engineer, it's our responsibility to help people as much as we can in all possible ways.**

Pneumonia is a very common disease. It can be either: 1) Bacterial pneumonia  2) Viral Pneumonia  3) Mycoplasma pneumonia   and 4) Fungal pneumonia.
This dataset consists pneumonia samples belonging to the first two classes.  The dataset consists of only very few samples and that too unbalanced. The aim of this kernel is to develop a robust deep learning model from scratch on this limited amount of data. We all know that deep learning models are data hungry but if you know how things work, you can build good models even with a limited amount of data. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import cv2
from keras import backend as K
color = sns.color_palette()
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Reproducibility is a great concern when doing deep learning. There was a good discussion on `KaggleNoobs` slack regarding this. We will set a numer of things in order to make sure that the results are almost reproducible(if not fully). 

In [ ]:
import tensorflow as tf

# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

# Set the numpy seed
np.random.seed(111)

# Disable multi-threading in tensorflow ops
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

# Set the random seed in tensorflow at graph level
tf.set_random_seed(111)

# Define a tensorflow session with above session configs
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)

# Set the session in keras
K.set_session(sess)

# Make the augmentation sequence deterministic
aug.seed(111)

The dataset is divided into three sets: 1) train set    2) validation set    and 3) test set.  Let's grab the dataset   

In [ ]:
# Define path to the data directory
#data_dir = Path('../input/chest-xray-pneumonia/chest_xray/chest_xray')
data_dir="abc"
# Path to train directory (Fancy pathlib...no more os.path!!)
#train_dir = data_dir / 'train'
train_dir="/kaggle/working/data_aug/train/content/data/content/FINAL_AUG_DATA/Train/"
# Path to validation directory
#val_dir = data_dir / 'val'
val_dir ="/kaggle/working/data_aug/val/content/data/content/FINAL_AUG_DATA/Val/"
# Path to test directory
#test_dir = data_dir / 'test'
test_dir = "/kaggle/working/data_aug/val/content/data/content/FINAL_AUG_DATA/Val/"

We will first go through the training dataset. We will do some analysis on that, look at some of the samples, check the number of samples for each class, etc.  Lets' do it.

Each of the above directory contains two sub-directories:
* `NORMAL`: These are the samples that describe the normal (no pneumonia) case.
* `VIRAL PNEUMONIA`: This directory contains those samples that are the pneumonia cases.
* `COVID-19`: This directory contains those samples that are the coronavirus cases.

In [ ]:
# Get the path to the normal and pneumonia sub-directories
from os.path import join
normal_cases_dir =join(train_dir,'NORMAL')
covid_cases_dir=join(train_dir,'COVID-19')
pneumonia_cases_dir = join(train_dir,'Viral Pneumonia')

print(normal_cases_dir)
# Get the list of all the images
"""
normal_cases = normal_cases_dir.glob('*')
pneumonia_cases = pneumonia_cases_dir.glob('*')
covid_cases = covid_cases_dir.glob('*')
"""
normal_cases = glob.glob(str(normal_cases_dir)+"/*")
pneumonia_cases = glob.glob(str(pneumonia_cases_dir)+"/*")
covid_cases = glob.glob(str(covid_cases_dir)+"/*")

# An empty list. We will insert the data into this list in (img_path, label) format
train_data = []

# Go through all the normal cases. The label for these cases will be 0
for img in normal_cases:
    train_data.append((img,0))

# Go through all the pneumonia cases. The label for these cases will be 1
for img in pneumonia_cases:
    train_data.append((img, 1))
    
# Go through all the coronavirus cases. The label for these cases will be 2
for img in covid_cases:
    train_data.append((img, 2))
    

# Get a pandas dataframe from the data we have in our list 
train_data = pd.DataFrame(train_data, columns=['image', 'label'],index=None)

# Shuffle the data 
train_data = train_data.sample(frac=1.).reset_index(drop=True)

# How the dataframe looks like?
train_data.head()

In [ ]:
train_data.shape

In [ ]:
9000*0.8

### How many samples for each class are there in the dataset?

In [ ]:
# Get the counts for each class
cases_count = train_data['label'].value_counts()
print(cases_count)

# Plot the results 
plt.figure(figsize=(10,8))
sns.barplot(x=cases_count.index, y= cases_count.values)
plt.title('Number of cases', fontsize=14)
plt.xlabel('Case type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(range(len(cases_count.index)), ['Normal(0)', 'Pneumonia(1)','Covid(2)'])
plt.show()

As you can see the data is highly imbalanced. We have almost with thrice pneumonia cases here as compared to the normal cases. This situation is very normal when it comes to medical data. The data will always be imbalanced. either there will be too many normal cases or there will be too many cases with the disease. 

Let's look at how a normal case is different from that of a pneumonia case. We will look at somes samples from our training data itself.

In [ ]:
# Get few samples for both the classes
pneumonia_samples = (train_data[train_data['label']==1]['image'].iloc[:5]).tolist()
normal_samples = (train_data[train_data['label']==0]['image'].iloc[:5]).tolist()
covid_samples = (train_data[train_data['label']==2]['image'].iloc[:5]).tolist()
# Concat the data in a single list and del the above two list
samples = pneumonia_samples + normal_samples+covid_samples
del pneumonia_samples, normal_samples,covid_samples

# Plot the data 
f, ax = plt.subplots(3,5, figsize=(30,15))
for i in range(15):
    img = imread(samples[i])
    ax[i//5, i%5].imshow(img, cmap='gray')
    if i<5:
        ax[i//5, i%5].set_title("Pneumonia")
    elif i<10:
        ax[i//5, i%5].set_title("Normal")
    else:
        ax[i//5, i%5].set_title("COVID-19")
    
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_aspect('auto')
plt.show()

If you look carefully, then there are some cases where you won't be able to differentiate between a normal case and a pneumonia case with the naked eye. There is one case in the above plot, at least for me ,which is too much confusing. If we can build a robust classifier, it would be a great assist to the doctor too.

### Preparing validation data
We will be defining a generator for the training dataset later in the notebook but as the validation data is small, so I can read the images and can load the data without the need of a generator.  This is exactly what the code block given below is doing.

In [ ]:
# Get the path to the sub-directories
normal_cases_dir = join(val_dir,'NORMAL')
pneumonia_cases_dir =join(val_dir,'Viral Pneumonia')
covid_cases_dir=join(val_dir,'COVID-19')

# Get the list of all the images
#normal_cases = normal_cases_dir.glob('*.jpeg')
#pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')



normal_cases = glob.glob(str(normal_cases_dir)+"/*")
pneumonia_cases = glob.glob(str(pneumonia_cases_dir)+"/*")
covid_cases = glob.glob(str(covid_cases_dir)+"/*")




# List that are going to contain validation images data and the corresponding labels
valid_data = []
valid_labels = []


# Some images are in grayscale while majority of them contains 3 channels. So, if the image is grayscale, we will convert into a image with 3 channels.
# We will normalize the pixel values and resizing all the images to 224x224 

# Normal cases
for img in normal_cases:
    img = cv2.imread(str(img))
    try:
        img = cv2.resize(img, (224,224),interpolation = cv2.INTER_AREA)
    except:
        raise ERR
        continue
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(0, num_classes=3)
    valid_data.append(img)
    valid_labels.append(label)
                      
# Pneumonia cases        
for img in pneumonia_cases:
    img = cv2.imread(str(img))
    try:
        img = cv2.resize(img, (224,224),interpolation = cv2.INTER_AREA)
    except:
        raise ERR
        continue
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(1, num_classes=3)
    valid_data.append(img)
    valid_labels.append(label)
    
    
# Coronavirus cases        
for img in covid_cases:
    img = cv2.imread(str(img))
    try:
        img = cv2.resize(img, (224,224),interpolation = cv2.INTER_AREA)
    except:
        raise ERR
        continue
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(2, num_classes=3)
    valid_data.append(img)
    valid_labels.append(label)
# Convert the list into numpy arrays
valid_data = np.array(valid_data)
valid_labels = np.array(valid_labels)

print("Total number of validation examples: ", valid_data.shape)
print("Total number of labels:", valid_labels.shape)

In [ ]:
9000*0.2

## Augmentation
Data augmentation is a powerful technique which helps in almost every case for improving the robustness of a model. But augmentation can be much more helpful where the dataset is imbalanced. You can generate different samples of undersampled class in order to try to balance the overall distribution. 

I like [imgaug](https://imgaug.readthedocs.io/en/latest/) a lot. It comes with a very clean api and you can do hell of augmentations with it. It's worth exploring!!
In the next code block, I will define a augmentation sequence. You will notice `Oneof` and it does exactly that. At each iteration, it will take one augmentation technique out of the three and will apply that on the samples 

In [ ]:
# Augmentation sequence 
seq = iaa.OneOf([
    #iaa.Fliplr(), # horizontal flips
    #iaa.Affine(rotate=20), # roatation
    iaa.Multiply((1.2, 1.5))]) #random brightness

### Training data generator 
Here I will define a very simple data generator. You can do more than this if you want but I think at this point, this is more than enough I need.

In [ ]:
def data_gen(data, batch_size):
    # Get total number of samples in the data
    n = len(data)
    steps = n//batch_size
    
    # Define two numpy arrays for containing batch data and labels
    batch_data = np.zeros((batch_size, 224, 224, 3), dtype=np.float32)
    batch_labels = np.zeros((batch_size,3), dtype=np.float32)

    # Get a numpy array of all the indices of the input data
    indices = np.arange(n)
    
    # Initialize a counter
    i =0
    while True:
        np.random.shuffle(indices)
        # Get the next batch 
        count = 0
        next_batch = indices[(i*batch_size):(i+1)*batch_size]
        for j, idx in enumerate(next_batch):
            img_name = data.iloc[idx]['image']
            label = data.iloc[idx]['label']
            
            # one hot encoding
            encoded_label = to_categorical(label, num_classes=3)
            # read the image and resize
            img = cv2.imread(str(img_name))
            img = cv2.resize(img, (224,224))
            
            # check if it's grayscale
            #if img.shape[2]==1:
            #    img = np.dstack([img, img, img])
            
            # cv2 reads in BGR mode by default
            orig_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # normalize the image pixels
            orig_img = img.astype(np.float32)/255.
            
            batch_data[count] = orig_img
            batch_labels[count] = encoded_label
            
            # generating more samples of the undersampled class
            if label==0 and count < batch_size-2:
            #    aug_img1 = seq.augment_image(img)
            #    aug_img2 = seq.augment_image(img)
            #    aug_img1 = cv2.cvtColor(aug_img1, cv2.COLOR_BGR2RGB)
            #    aug_img2 = cv2.cvtColor(aug_img2, cv2.COLOR_BGR2RGB)
            #    aug_img1 = aug_img1.astype(np.float32)/255.
            #    aug_img2 = aug_img2.astype(np.float32)/255.

            #    batch_data[count+1] = aug_img1
            #    batch_labels[count+1] = encoded_label
            #    batch_data[count+2] = aug_img2
            #    batch_labels[count+2] = encoded_label
                count +=2
            
            else:
                count+=1
            
            if count==batch_size-1:
                break
            
        i+=1
        yield batch_data, batch_labels
            
        if i>=steps:
            i=0

## Model 

This is the best part. If you look at other kernels on this dataset, everyone is busy doing transfer learning and fine-tuning. **You should transfer learn but wisely**.  We will be doing partial transfer learning and rest of the model will be trained from scratch. I will explain this in detail but before that, I would love to share one of the best practices when it comes to building deep learning models from scratch on limited data.

1. Choose a simple architecture.
2. Initialize the first few layers from a network that is pretrained on imagenet. This is because first few layers capture general details like color blobs, patches, edges, etc. Instead of randomly initialized weights for these layers, it would be much better if you fine tune them.
3. Choose layers that introduce a lesser number of parameters. For example, `Depthwise SeparableConv` is a good replacement for `Conv` layer. It introduces lesser number of parameters as compared to normal convolution and as different filters are applied to each channel, it captures more information.  `Xception` a powerful network, is built on top of such layers only. You can read about `Xception` and `Depthwise Separable Convolutions` in [this](https://arxiv.org/abs/1610.02357) paper.
4. Use batch norm with convolutions.  As the network becomes deeper, batch norm start to play an important role.
5. Add dense layers with reasonable amount of neurons. Train with a higher learning rate and experiment with the number of neurons in the dense layers. Do it for the depth of your network too. 
6. Once you know a good depth, start training your network with a lower learning rate along with decay.  

This is all that I have done in the next code block.


In [ ]:
def build_model2():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv0_1')(input_img)
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv0_2')(x)
    x = MaxPooling2D((2,2), name='pool0')(x)
    
    x = SeparableConv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(x)
    x = SeparableConv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(3, activation='softmax', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model

In [ ]:
def build_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(3, activation='softmax', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model

In [ ]:
model =  build_model()
model.summary()

We will initialize the weights of first two convolutions with imagenet weights,

In [ ]:
# Open the VGG16 weight file
f = h5py.File('../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'r')

# Select the layers for which you want to set weight.

w,b = f['block1_conv1']['block1_conv1_W_1:0'], f['block1_conv1']['block1_conv1_b_1:0']
model.layers[1].set_weights = [w,b]

w,b = f['block1_conv2']['block1_conv2_W_1:0'], f['block1_conv2']['block1_conv2_b_1:0']
model.layers[2].set_weights = [w,b]

w,b = f['block2_conv1']['block2_conv1_W_1:0'], f['block2_conv1']['block2_conv1_b_1:0']
model.layers[4].set_weights = [w,b]

w,b = f['block2_conv2']['block2_conv2_W_1:0'], f['block2_conv2']['block2_conv2_b_1:0']
model.layers[5].set_weights = [w,b]

f.close()
model.summary()    

In [ ]:
# opt = RMSprop(lr=0.0001, decay=1e-6)
#change the optimizer Adam to SGD with lr = .1
#labels converted to categorical?
opt = Adam(lr=0.0001, decay=1e-5)
#from keras_adabound import AdaBound
#loss=tf.keras.losses.CategoricalCrossentropy()

#model.compile(optimizer=AdaBound(lr=1e-3, final_lr=0.1), loss=model_loss)
#opt = Adam(lr=0.1, decay=1e-2)
es = EarlyStopping(patience=5)
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
#model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)
#metrics=['accuracy', 'categorical_accuracy', 'precision', 'recall']

In [ ]:
batch_size = 16
nb_epochs = 3

# Get a train data generator
train_data_gen = data_gen(data=train_data, batch_size=batch_size)

# Define the number of training steps
nb_train_steps = train_data.shape[0]//batch_size

print("Number of training and validation steps: {} and {}".format(nb_train_steps, len(valid_data)))

I have commented out the training step as of now as it will train the network again while rendering the notebook and I would have to wait for an hour or so which I don't want to. I have uploaded the weights of the best model I achieved so far. Feel free to use it for further fine-tuning of the network. We will load those weights and will run the inference on the test set using those weights only. But...but for your reference, I will attach the screenshot of the training steps here.

In [ ]:
# Fit the model
history = model.fit_generator(train_data_gen, epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                              validation_data=(valid_data, valid_labels),callbacks=[es, chkpt],
                              class_weight='balanced')

In [ ]:
model.save("mymodel.h5")

In [ ]:
# from IPython.display import FileLink
# FileLink(r'best_model_todate')

In [ ]:
#  os.path.getsize("best_model_todate")/(1024*1024) #megabytes

In [ ]:
os.listdir("/kaggle/working")

In [ ]:
# Load the model weights
# model.load_weights("../input/xray-best-model/best_model/best_model.hdf5")
# model.load_weights("/kaggle/input/xray-best-model/best_model.hdf5")
model.load_weights("/kaggle/working/mymodel.h5")

In [ ]:
test_dir

In [ ]:
# Preparing test data
normal_cases_dir = join(test_dir,'NORMAL')
pneumonia_cases_dir = join(test_dir,'Viral Pneumonia')
covid_cases_dir=join(test_dir,'COVID-19')

normal_cases = glob.glob(str(normal_cases_dir) + "/*")
pneumonia_cases = glob.glob(str(pneumonia_cases_dir) + "/*")
covid_cases = glob.glob(str(covid_cases_dir)+"/*")

test_data = []
test_labels = []

for img in normal_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(0, num_classes=3)
    test_data.append(img)
    test_labels.append(label)
                      
for img in pneumonia_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(1, num_classes=3)
    test_data.append(img)
    test_labels.append(label)
    
for img in covid_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(2, num_classes=3)
    test_data.append(img)
    test_labels.append(label)

test_data = np.array(test_data)
test_labels = np.array(test_labels)

print("Total number of test examples: ", test_data.shape)
print("Total number of labels:", test_labels.shape)



In [ ]:
# Evaluation on test dataset
test_loss, test_score = model.evaluate(test_data, test_labels, batch_size=16)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

In [ ]:
# Get predictions
preds = model.predict(test_data, batch_size=16)
preds = np.argmax(preds, axis=-1)

# Original labels
orig_test_labels = np.argmax(test_labels, axis=-1)

print(orig_test_labels.shape)
print(preds.shape)

When a particular problem includes an imbalanced dataset, then accuracy isn't a good  metric to look for. For example, if your dataset contains 95 negatives and 5 positives, having a model with 95% accuracy doesn't make sense at all. The classifier might label every example as negative and still achieve 95% accuracy. Hence,  we need to look for alternative metrics. **Precision** and **Recall** are really good metrics for such kind of problems. 

We will get the confusion matrix from our predictions and see what is the recall and precision of our model.

In [ ]:
# Get the confusion matrix
cm  = confusion_matrix(orig_test_labels, preds)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True, alpha=0.8,cmap=plt.cm.Blues)
plt.xticks(range(3), ['Normal', 'Pneumonia', 'Corona'], fontsize=16)
plt.yticks(range(3), ['Normal', 'Pneumonia', 'Corona'], fontsize=16)
plt.show()

In [ ]:
cm

In [ ]:
def confusion_matrix_for(cls, cm):
    TP = cm[cls, cls]
    FN = cm[cls].sum() - TP
    FP = cm[:, cls].sum() - TP
    TN = cm.sum() - TP - FN - FP
    return np.array([[TP, FN], [FP, TN]])

In [ ]:
confusion_matrix_for(0, cm).ravel()

In [ ]:
confusion_matrix_for(1, cm).ravel()

In [ ]:
confusion_matrix_for(2, cm).ravel()

In [ ]:
for cls in range(cm.shape[0]):
    print(f'[Class {cls} vs others]')
    TP, FN, FP, TN = confusion_matrix_for(cls, cm).ravel()
    print(f'TP: {TP}, FN: {FN}, FP: {FP}, TN: {TN}')
    # compute your metrics (your code in the question)
    print()

In [ ]:
# Calculate Precision and Recall for class 0
TN, FP, FN, TP = confusion_matrix_for(0, cm).ravel()
precision_0 = TP/(TP+FP)
recall_0 = TP/(TP+FN)

print("Recall of the model is {:.2f}".format(recall_0))
print("Precision of the model is {:.2f}".format(precision_0))

In [ ]:
# Calculate Precision and Recall for class 1
TN, FP, FN, TP = confusion_matrix_for(1, cm).ravel()
precision_1 = TP/(TP+FP)
recall_1 = TP/(TP+FN)

print("Recall of the model is {:.2f}".format(recall_1))
print("Precision of the model is {:.2f}".format(precision_1))

In [ ]:
# Calculate Precision and Recall for class 2
TN, FP, FN, TP = confusion_matrix_for(2, cm).ravel()
precision_2 = TP/(TP+FP)
recall_2 = TP/(TP+FN)

print("Recall of the model is {:.2f}".format(recall_2))
print("Precision of the model is {:.2f}".format(precision_2))

In [ ]:
from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()


with open("model_2.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_2.h5")

In [ ]:
os.listdir("/kaggle/working")

Nice!!! So, our model has a 98% recall. In such problems, a good recall value is expected. But if you notice, the precision is only 80%. This is one thing to notice. Precision and Recall follows a trade-off, and you need to find a point where your recall, as well as your precision, is more than good but both can't increase simultaneously. 

That's it folks!! I hope you enjoyed this kernel. Happy Kaggling!!